## Query for the weather data for a day and hour in a given region

In [11]:
import argparse
import datetime as dt
import sys

import meteomatics.api as api

In [12]:
###Credentials:
username = 'hackz_bucher_3'
password = 'GbIj6UqbTi5C'

In [13]:
# startdate_station_ts = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)-dt.timedelta(days=0)
# enddate_station_ts = startdate_station_ts + dt.timedelta(days=2)
# interval_station_ts = dt.timedelta(hours=1)

In [14]:
startdate_station_ts = dt.datetime(2019,4,1,0,0) # 1 April
# enddate_station_ts = dt.datetime(2019,4,10,0,0) # 26 Sept
enddate_station_ts = dt.datetime(2019,9,26,0,0) # 26 Sept
interval_station_ts = dt.timedelta(hours=1)

#### Human Thermal comfort factors

- Temperature: 2 metres above ground in Celcius 't_2m:C'
- Relative humidity: 2 metres above ground in pascals 'relative_humidity_2m:p' --> NA
- Median Radiant temperature --> NA
- Wind speed: 10 metres above ground in meter per sec 'wind_speed_10m:ms'

#### Possibility of rain
- Precipiation: In millimeters that has fallen in one hour 'precip_1h:mm'
- Cloud cover: effective Cloud cover in Octas 'effective_cloud_cover:octas' --> NA
- Thunderstorm probability: Probability of thunderstorm for the next 24 hours 'prob_tstorm_24h' --> NA

In [15]:
parameters_station_ts = ['t_2m:C', 'wind_speed_10m:ms', 'precip_1h:mm']#,'relative_humidity_2m:p','effective_cloud_cover:octas', 'prob_tstorm_24h'

In [16]:
model_station_ts = 'mix-obs' # 'mix' for prediction and 'mix-obs' for observation

In [17]:
coordinates_station_ts = [(47.532477, 7.601737973795679)] # Coordinates of Basel

In [18]:
print("station coordinates timeseries:")
try:
    df_sd_coord = api.query_station_timeseries(startdate_station_ts, enddate_station_ts, interval_station_ts,
                                               parameters_station_ts, username, password, model=model_station_ts,
                                               latlon_tuple_list=coordinates_station_ts, on_invalid='fill_with_invalid',
                                               request_type="POST", temporal_interpolation='none')
    print(df_sd_coord.head())
except Exception as e:
    print("Failed, the exception is {}".format(e))

station coordinates timeseries:
2019-09-28 12:07:10| INFO |Calling URL: https://api.meteomatics.com/2019-04-01T00:00:00+00:00--2019-09-26T00:00:00+00:00:PT1H (username = hackz_bucher_3)
                                                       t_2m:C  \
station_id                  validdate                           
47.532477,7.601737973795679 2019-04-01 00:00:00+00:00     8.0   
                            2019-04-01 01:00:00+00:00     7.0   
                            2019-04-01 02:00:00+00:00     6.3   
                            2019-04-01 03:00:00+00:00     5.1   
                            2019-04-01 04:00:00+00:00     4.8   

                                                       wind_speed_10m:ms  \
station_id                  validdate                                      
47.532477,7.601737973795679 2019-04-01 00:00:00+00:00                1.5   
                            2019-04-01 01:00:00+00:00                2.4   
                            2019-04-01 02:00:00+00:00 

In [19]:
maximum_temperature = df_sd_coord['t_2m:C'].max()
# minimum_windspeed = df_sd_coord['wind_speed_10m:ms'].min()
mean_temperature = df_sd_coord['t_2m:C'].mean()
just_precipitation = df_sd_coord['precip_1h:mm']
first_value_of_precipitation = df_sd_coord.iloc[0]['precip_1h:mm']
just_at_12UTC = df_sd_coord[df_sd_coord.index.get_level_values('validdate').hour == 12]

In [27]:
df_sd_coord[0:24]['t_2m:C']

station_id                   validdate                
47.532477,7.601737973795679  2019-04-01 00:00:00+00:00     8.0
                             2019-04-01 01:00:00+00:00     7.0
                             2019-04-01 02:00:00+00:00     6.3
                             2019-04-01 03:00:00+00:00     5.1
                             2019-04-01 04:00:00+00:00     4.8
                             2019-04-01 05:00:00+00:00     4.3
                             2019-04-01 06:00:00+00:00     4.5
                             2019-04-01 07:00:00+00:00     7.0
                             2019-04-01 08:00:00+00:00     8.3
                             2019-04-01 09:00:00+00:00    10.4
                             2019-04-01 10:00:00+00:00    12.1
                             2019-04-01 11:00:00+00:00    12.9
                             2019-04-01 12:00:00+00:00    15.4
                             2019-04-01 13:00:00+00:00    16.2
                             2019-04-01 14:00:00+00:00    17.0


In [27]:
export_csv = df_sd_coord.to_csv (r'./weather_data.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path

## Get max,min,mean values of temp

In [111]:
import pandas as pd
my_csv = pd.read_csv('./weather_data.csv')

In [112]:
output = pd.DataFrame()
temp_dict = dict()
for i in range(0,(df_sd_coord.shape[0]-1),24):
    temp_dict['validdate']=dateutil.parser.parse(my_csv['validdate'][i])
    temp_dict['mean_temp']=my_csv['t_2m:C'][i:i+24].mean()
    temp_dict['max_temp']=my_csv['t_2m:C'][i:i+24].max()
    temp_dict['min_temp']=my_csv['t_2m:C'][i:i+24].min()
    temp_dict['total_prep']=my_csv['precip_1h:mm'][i:i+24].sum()
    temp_dict['mean_wind_speed']=my_csv['wind_speed_10m:ms'][i:i+24].mean()
    output = output.append(temp_dict, ignore_index=True)

In [113]:
export_csv = output.to_csv (r'./weather_data_updated.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path